In [ ]:
import os

import pandas as pd
from sleeper_wrapper import League, Players, Stats

from draft_optimizer.src.utils import DATA_DIR

In [ ]:
# Specify constants
LEAGUE_ID = "837802583950209024"
YEAR = 2022

# Get Players

In [ ]:
# Get players
players_raw = Players().get_all_players()
players_df = pd.DataFrame.from_dict(players_raw).T

In [ ]:
# Subset and rename columns
info_cols = ["full_name", "active", "team"]
pos_cols = ["position", "fantasy_positions"]
id_cols = [c for c in players_df.columns if "id" in c]
keep_cols = info_cols + pos_cols + id_cols
players = players_df[keep_cols].rename({"player_id": "sleeper_id"}, axis=1).set_index("sleeper_id")

# Fix defenses
is_def = players["position"] == "DEF"
players.loc[is_def, "position"] = "D/ST"
players.loc[is_def, "fantasy_positions"] = "[D/ST]"
players.loc[is_def, "full_name"] = players_df.loc[is_def, "last_name"] + " D/ST"
# TODO: other league IDs?

print(players.shape)
players.head()

In [ ]:
# Output
out_dir = os.path.join(DATA_DIR, "sleeper", str(YEAR))
if not os.path.isdir(out_dir):
    Path(out_dir).mkdir(parents=True, exist_ok=True)
players.to_csv(os.path.join(out_dir, "players.csv"), index=True)

# Get Projections

In [ ]:
# Get regular season projections
weeks = list(range(1, 18 + 1))  # 1-18
week_proj_dfs = []
for week in weeks:
    # Get data
    week_proj_raw = Stats().get_week_projections("regular", YEAR, week)
    week_proj_df = pd.DataFrame.from_dict(week_proj_raw).T
    week_proj_df.index.name = "sleeper_id"

    # Append
    week_proj_dfs.append(week_proj_df)

In [ ]:
# Aggregate projections
pts_cols = ["pts_std", "pts_ppr", "pts_half_ppr"]
proj_dfs = []
for i, week_proj_df in enumerate(week_proj_dfs):
    # Subset
    week = i + 1
    week_proj_df = week_proj_df[pts_cols].copy()
    week_proj_df.columns = pd.MultiIndex.from_product([[f"week{week}"], week_proj_df.columns])

    # Append
    proj_dfs.append(week_proj_df)

In [ ]:
# Concat projections
projs = pd.concat(proj_dfs, axis=1)

# Export point projections
for pts_col in pts_cols:
    # Prepare data
    projs_pts = projs.loc[:, projs.columns.get_level_values(1) == pts_col].copy()
    projs_pts.columns = projs_pts.columns.get_level_values(0)
    projs_pts["sum_weeks"] = projs_pts.sum(axis=1)

    # Output
    out_dir = os.path.join(DATA_DIR, "sleeper", str(YEAR))
    if not os.path.isdir(out_dir):
        Path(out_dir).mkdir(parents=True, exist_ok=True)
    projs_pts.to_csv(os.path.join(out_dir, f"projections_{pts_col.replace('pts_', '')}.csv"), index=True)

# Misc.

In [ ]:
# Get league
league = League(LEAGUE_ID)

In [ ]:
# Get users
users = league.get_users()

# View users
for user in users:
    print(user["display_name"], user["user_id"])

In [ ]:
# Get draft
league.get_league()